In [1]:
# Welcome to RAD-MMM inference tutorial

## imports
import pytorch_lightning as pl
import sys
import yaml
sys.path.append('/akshit/scratch/RAD-MMM/vocoders')
sys.path.append('/akshit/scratch/RAD-MMM')
from pytorch_lightning.cli import LightningCLI
from tts_lightning_modules import TTSModel
from data_modules import BaseAudioDataModule
from jsonargparse import lazy_instance
from decoders import RADMMMFlow
from loss import RADTTSLoss
import inspect
from pytorch_lightning.callbacks import ModelCheckpoint
from training_callbacks import LogDecoderSamplesCallback, \
    LogAttributeSamplesCallback
from utils import get_class_args
from tts_text_processing.text_processing import TextProcessing
from common import Encoder

In [2]:
from tts_main import RADTTSLightningCLI,lcli

In [3]:
pl.__version__

'2.3.0dev'

In [4]:
# Set paths
attribute_model_path = "/akshit/scratch/generator_ckpt/radmmm_public/attribute_model.ckpt"
decoder_model_path = "/akshit/scratch/generator_ckpt/radmmm_public/decoder.ckpt"
gen_config_path = "/akshit/scratch/generator_ckpt/radmmm_public/config_interactive.yaml"
voc_model_path = "/akshit/scratch/generator_ckpt/hfg_public/g_00072000"
voc_config_path = "/akshit/scratch/generator_ckpt/hfg_public/config_16khz.json"
phonemizer_cfg='{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}'

In [5]:
# import mock

# with mock.patch("sys.argv", ["predict", "--config="+gen_config_path,\
#                              # "--ckpt_path="+attribute_model_path,\
#                              "--model.predict_mode=tts", \
#                              "--data.inference_transcript=model_inputs/resynthesis_prompts.json", \
#                              "--model.prediction_output_dir=/akshit/scratch/RAD-MMM/tutorials/out1", \
#                              "--trainer.devices=1", "--data.batch_size=1", \
#                              "--model.vocoder_checkpoint_path="+voc_model_path, \
#                              "--model.vocoder_config_path="+voc_config_path, \
#                              "--data.phonemizer_cfg="+'{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}', \
#                              "--model.encoders_path="+decoder_model_path, \
#                              "--model.decoder_path="+decoder_model_path, \
#                              "--model.output_directory=/akshit/scratch/RAD-MMM/tutorials/run1"]):
#     cli = RADTTSLightningCLI(TTSModel, BaseAudioDataModule, save_config_kwargs={"overwrite": True}, run=False)

In [6]:
# results in issue - SIGSEV

# cli.trainer.predict(model=cli.model,datamodule=cli.datamodule)

In [5]:
# attempting to load the model
import pytorch_lightning
import inspect
inspect.signature(pytorch_lightning.cli.instantiate_module())

TypeError: instantiate_module() missing 2 required positional arguments: 'class_type' and 'config'

In [7]:
# https://github.com/Lightning-AI/pytorch-lightning/pull/18105/files 
model2= TTSModel.load_from_checkpoint(checkpoint_path=attribute_model_path,hparams_file="/akshit/scratch/RAD-MMM/tutorials/run1/lightning_logs/version_13/hparams.yaml", strict=False)


ArgumentError: Parser key "module.decoder":
  Not a valid subclass of Module. Got value: {'class_path': 'decoders.RADMMMFlow', 'init_args': {'n_speaker_dim': 16, 'use_accent': True, 'n_accent_dim': 8, 'n_text_dim': 520, 'n_group_size': 2, 'n_mel_channels': 80, 'use_spk_emb_for_alignment': False, 'n_f0_dims': 1, 'n_energy_avg_dims': 1, 'context_w_f0_and_energy': True, 'use_context_lstm': True, 'n_flows': 8, 'n_conv_layers_per_step': 4, 'n_early_size': 2, 'n_early_every': 2, 'affine_model': 'wavenet', 'scaling_fn': 'tanh', 'affine_activation': 'softplus', 'use_partial_padding': True, 'n_splines': 0, 'use_bn': True, 'freeze_whitening_layer': False, 'use_accent_emb_for_decoder': False}}
  Subclass types expect one of:
  - a class path (str)
  - a dict with class_path entry
  - a dict without class_path but with init_args entry (class path given previously)

In [ ]:
## some runtime error
# with open("/akshit/scratch/RAD-MMM/tutorials/run1/lightning_logs/version_12/hparams.yaml", "r") as f:
#     hparams = yaml.safe_load(f)
    
# model2 = pytorch_lightning.cli.instantiate_module(TTSModel,config=hparams)


In [ ]:
import torch
model3=torch.load(attribute_model_path)

In [ ]:
model3

In [ ]:
model4 = torch.load(decoder_model_path)

In [ ]:
model4

In [ ]:
# run=True, like cmd line; https://lightning.ai/docs/pytorch/stable/cli/lightning_cli_advanced_3.html, fails with SIGSEGV
# inp = ["predict", "--config="+gen_config_path,\
#          # "--ckpt_path="+attribute_model_path,\
#          "--model.predict_mode=tts", \
#          "--data.inference_transcript='model_inputs/resynthesis_prompts.json'", \
#          "--model.prediction_output_dir='/akshit/scratch/RAD-MMM/tutorials/out1'", \
#          "--trainer.devices=1", "--data.batch_size=1", \
#          "--model.vocoder_checkpoint_path="+voc_model_path, \
#          "--model.vocoder_config_path="+voc_config_path, \
#          "--data.phonemizer_cfg="+'{"en_US": "assets/en_US_word_ipa_map.txt","es_MX": "assets/es_MX_word_ipa_map.txt","de_DE": "assets/de_DE_word_ipa_map.txt","en_UK": "assets/en_UK_word_ipa_map.txt","es_CO": "assets/es_CO_word_ipa_map.txt","es_ES": "assets/es_ES_word_ipa_map.txt","fr_FR": "assets/fr_FR_word_ipa_map.txt","hi_HI": "assets/hi_HI_word_ipa_map.txt","pt_BR": "assets/pt_BR_word_ipa_map.txt","te_TE": "assets/te_TE_word_ipa_map.txt"}', \
#          "--model.encoders_path="+decoder_model_path, \
#          "--model.decoder_path="+decoder_model_path, \
#          "--model.output_directory='/akshit/scratch/RAD-MMM/tutorials/run1'"]
# lcli(inp)

In [ ]:
# https://github.com/Lightning-AI/pytorch-lightning/pull/18105 

In [ ]:
# # load the config files

# with open(gen_config_path, "r") as f:
#     gen_config = yaml.safe_load(f)
    
#     # set config
#     gen_config["ckpt_path"] = attribute_model_path
#     gen_config["model"]["predict_mode"]="tts"
#     gen_config["model"]["prediction_output_dir"]="/akshit/scratch/RAD-MMM/tutorials/out1" #ToDo
#     gen_config["model"]["vocoder_checkpoint_path"]=voc_model_path
#     gen_config["model"]["vocoder_config_path"]=voc_model_path
#     gen_config["model"]["decoder_path"]=decoder_model_path
#     gen_config["model"]["encoders_path"]=decoder_model_path
#     gen_config["model"]["output_directory"]="/akshit/scratch/RAD-MMM/tutorials/run1"
#     gen_config["trainer"]["devices"]=1
#     gen_config["data"]["inference_transcript"]="model_inputs/resynthesis_prompts.json" #ToDo
#     gen_config["data"]["batch_size"]=1
#     gen_config["data"]["phonemizer_cfg"]=phonemizer_cfg
    
#     # set defaults
#     gen_config["checkpoint_callback"]["filename"]="latest-epoch_{epoch}-iter_{global_step:.0f}"
#     gen_config["checkpoint_callback"]["monitor"]="global_step"
#     gen_config["checkpoint_callback"]["mode"]= "max"
#     gen_config["checkpoint_callback"]["every_n_train_steps"]= 3000
#     gen_config["checkpoint_callback"]["dirpath"]= "/debug"
#     gen_config["checkpoint_callback"]["save_top_k"] = -1
#     gen_config["checkpoint_callback"]["auto_insert_metric_name"]=False
    
#     # linking args
#     gen_config["model"]["phonemizer_cfg"]=gen_config["data"]["phonemizer_cfg"]
#     gen_config["model"]["add_bos_eos_to_text"]=gen_config["data"]["add_bos_eos_to_text"]
#     gen_config["model"]["append_space_to_text"]=gen_config["data"]["append_space_to_text"]
#     gen_config["model"]["prepend_space_to_text"]=gen_config["data"]["prepend_space_to_text"]
#     gen_config["model"]["handle_phoneme_ambiguous"]=gen_config["data"]["handle_phoneme_ambiguous"]
#     gen_config["model"]["handle_phoneme"]=gen_config["data"]["handle_phoneme"]
#     gen_config["model"]["p_phoneme"]=gen_config["data"]["p_phoneme"]
#     gen_config["model"]["phoneme_dict_path"]=gen_config["data"]["phoneme_dict_path"]
#     gen_config["model"]["heteronyms_path"]=gen_config["data"]["heteronyms_path"]
#     gen_config["model"]["cleaner_names"]=gen_config["data"]["cleaner_names"]
#     gen_config["model"]["symbol_set"]=gen_config["data"]["symbol_set"]
#     gen_config["model"]["sampling_rate"]=gen_config["data"]["sampling_rate"]
#     gen_config["checkpoint_callback"]["dirpath"]=gen_config["model"]["output_directory"]
#     gen_config["trainer"]["default_root_dir"]=gen_config["model"]["output_directory"]
    
    

In [ ]:
model = cli.model
data_module = cli.datamodule
trainer = cli.trainer

In [ ]:
predictions = trainer.predict(model=model)

In [ ]:
model = config_init.model
data_module = config_init.data
trainer = config_init.trainer

In [ ]:
# first we go through one input at a time

In [ ]:
# resynthesis input
resyn_input = {
      "script": "{ʈ ˈɪ h ə ɾ ˌi} {b ˈã dʰ} {b ˈʌ n n ˌeː} {k eː} {d ɔː ɾ ˈaː n} {s ˈʌ n t oː} {n ˈeː} {ʋ ɪ ɾ ˈoː dʰ} {ɟ ə ɾ ˈuː ɾ} {k ˈɪ j aː} {l ˈeː k ɪ n} {pʰ ˈɪ ɾ} {tʰ ˈʌ m} {ɡ ˈʌ eː}",
      "spk_id": "indic-iiit-hyd-female",
      "decoder_spk_id": "indic-iiit-hyd-female",
      "duration_spk_id": "indic-iiit-hyd-female",
      "energy_spk_id": "indic-iiit-hyd-female",
      "f0_spk_id": "indic-iiit-hyd-female",
      "language": "hi_HI",
      "emotion": "other"}

In [ ]:
# here is a gradio interface to try out multiple combinations